## 1. Постановка задачи

<center> <img src=https://storage.googleapis.com/kaggle-competitions/kaggle/3333/media/taxi_meter.png align="right" width="300"/> </center>
    
Вам предстоит решить настоящую задачу машинного обучения, направленную на автоматизацию бизнес процессов. Мы построим модель, которая будет предсказывать общую продолжительность поездки такси в Нью-Йорке. 

Представьте вы заказываете такси из одной точки Нью-Йорка в другую, причем не обязательно конечная точка должна находиться в пределах города. Сколько вы должны будете за нее заплатить? Известно, что стоимость такси в США  рассчитывается на основе фиксированной ставки + тарифная стоимость, величина которой зависит от времени и расстояния. Тарифы варьируются в зависимости от города.

В свою очередь время поездки зависит от множества факторов таких как, откуда и куда вы едете, в какое время суток вы совершаете вашу поездку, погодных условий и так далее. 

Таким образом, если мы разработаем алгоритм, способный определять длительность поездки, мы сможем прогнозировать ее стоимость самым тривиальным образом, например, просто умножая стоимость на заданный тариф. 
Сервисы такси хранят огромные объёмы информации о поездках, включая такие данные как конечная, начальная точка маршрута, дата поездки и ее длительность. Эти данные можно использовать для того, чтобы прогнозировать длительность поездки в автоматическом режиме с привлечением искусственного интеллекта.

**Бизнес-задача:** определить характеристики и с их помощью спрогнозировать длительность поездки такси.

**Техническая задача для вас как для специалиста в Data Science:** построить модель машинного обучения, которая на основе предложенных характеристик клиента будет предсказывать числовой признак - время поездки такси. То есть решить задачу регрессии.

**Основные цели проекта:**
1. Сформировать набор данных на основе нескольких источников информации
2. Спроектировать новые признаки с помощью Feature Engineering и выявить наиболее значимые при построении модели
3. Исследовать предоставленные данные и выявить закономерности
4. Построить несколько моделей и выбрать из них наилучшую по заданной метрике
5. Спроектировать процесс предсказания времени длительности поездки для новых данных

Загрузить свое решение на платформу Kaggle, тем самым поучаствовав в настоящем Data Science соревновании.
Во время выполнения проекта вы отработаете навыки работы с несколькими источниками данных, генерации признаков, разведывательного анализа и визуализации данных, отбора признаков и, конечно же, построения моделей машинного обучения!


## 2. Знакомство с данными, базовый анализ и расширение данных

Начнём наше исследование со знакомства с предоставленными данными. А также подгрузим дополнительные источники данных и расширим наш исходный датасет. 


Заранее импортируем модули, которые нам понадобятся для решения задачи:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection

Прочитаем наш файл с исходными данными:

In [ ]:
taxi_data = pd.read_csv("train.csv")
print('Train data shape: {}'.format(taxi_data.shape))
taxi_data.head()

Итак, у нас с вами есть данные о почти 1.5 миллионах поездок и 11 характеристиках, которые описывают каждую из поездок. 

Мы условно разделили признаки нескольких групп. Каждой из групп мы в дальнейшем уделим отдельное внимание.

**Данные о клиенте и таксопарке:**
* id - уникальный идентификатор поездки
* vendor_id - уникальный идентификатор поставщика (таксопарка), связанного с записью поездки

**Временные характеристики:**
* pickup_datetime - дата и время, когда был включен счетчик поездки
* dropoff_datetime - дата и время, когда счетчик был отключен

**Географическая информация:**
* pickup_longitude -  долгота, на которой был включен счетчик
* pickup_latitude - широта, на которой был включен счетчик
* dropoff_longitude - долгота, на которой счетчик был отключен
* dropoff_latitude - широта, на которой счетчик был отключен

**Прочие признаки:**
* passenger_count - количество пассажиров в транспортном средстве (введенное водителем значение)
* store_and_fwd_flag - флаг, который указывает, сохранилась ли запись о поездке в памяти транспортного средства перед отправкой поставщику. Y - хранить и пересылать, N - не хранить и не пересылать поездку.

**Целевой признак:**
* trip_duration - продолжительность поездки в секундах


Для начала мы проведем базовый анализ того, насколько данные готовы к дальнейшей предобработке и анализу. 

### Задание 2.1
Для начала посмотрим на временные рамки, в которых мы работаем с данными.

Переведите признак pickup_datetime в тип данных datetime с форматом год-месяц-день час:минута:секунда (в функции pd.to_datetime() параметр format='%Y-%m-%d %H:%M:%S'). 

Определите временные рамки (без учета времени), за которые представлены данные.

In [ ]:
taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
taxi_data['pickup_datetime'].min(), taxi_data['pickup_datetime'].max()

In [ ]:
taxi_data['dropoff_datetime'] = pd.to_datetime(taxi_data['dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
taxi_data['val_time'] = taxi_data['dropoff_datetime'] - taxi_data['pickup_datetime']

In [ ]:
taxi_data.describe()

In [ ]:
import datetime

# Создаем объект timedelta с указанным значением
delta = datetime.timedelta(days=40, hours=19, minutes=31, seconds=22)

# Получаем общее количество секунд, используя атрибут total_seconds()
total_seconds = delta.total_seconds()

print(total_seconds)

In [ ]:
# Создаем объект timedelta с указанным значением
delta = datetime.timedelta(days=0, hours=0, minutes=11, seconds=2)

# Получаем общее количество секунд, используя атрибут total_seconds()
total_seconds = delta.total_seconds()

print(total_seconds)

### Задание 2.2
Посмотрим на пропуски. 
Сколько пропущенных значений присутствует в данных (суммарно по всем столбцам таблицы)?

In [ ]:
taxi_data.info()

In [ ]:
taxi_data.isnull().sum()

### Задание 2.3
Посмотрим на статистические характеристики некоторых признаков. 

а) Сколько уникальных таксопарков присутствует в данных?

б) Каково максимальное количество пассажиров?

в) Чему равна средняя и медианная длительность поездки? Ответ приведите в секундах и округлите до целого.

г) Чему равно минимальное и максимальное время поездки (в секундах)?


In [ ]:
taxi_data['vendor_id'].value_counts()

In [ ]:
taxi_data['passenger_count'].value_counts()

Займемся расширением исходного набора данных как с помощью внешних источников, так и с помощью манипуляций над имеющимися в данных признаками. 


### Задание 2.4
Реализуйте функцию add_datetime_features(), которая принимает на вход таблицу с данными о поездках (DataFrame) и возвращает ту же таблицу с добавленными в нее 3 столбцами:
* pickup_date - дата включения счетчика - начала поездки (без времени);
* pickup_hour - час дня включения счетчика;
* pickup_day_of_week - наименование дня недели, в который был включен счетчик.

а) Сколько поездок было совершено в субботу?

б) Сколько поездок в среднем совершается в день? Ответ округлите до целого

In [ ]:
def add_datetime_features(DataFrame):
    DataFrame['pickup_date'] = pd.to_datetime(DataFrame['pickup_datetime'], format='%Y-%m-%d %H:%M:%S').dt.date
    DataFrame['pickup_hour'] = pd.to_datetime(DataFrame['pickup_datetime'], format='%Y-%m-%d %H:%M:%S').dt.hour
    # DataFrame['pickup_day_of_week'] = pd.to_datetime(DataFrame['pickup_datetime'], format='%Y-%m-%d %H:%M:%S').dt.day_of_week
    DataFrame['pickup_day_of_week'] = pd.to_datetime(DataFrame['pickup_datetime'], format='%Y-%m-%d %H:%M:%S').dt.day_name()
    return DataFrame


In [ ]:
taxi_data.head()

In [ ]:


add_datetime_features(taxi_data)
# print (taxi_data[taxi_data['pickup_day_of_week'] == 5].shape[0])
print (taxi_data[taxi_data['pickup_day_of_week'] == 'Saturday'].shape[0])



In [ ]:


# print (taxi_data['pickup_date'].shape[0]/len(taxi_data['pickup_date'].unique()))
print (taxi_data['pickup_date'].shape[0]/taxi_data['pickup_date'].unique().shape[0])



### Задание 2.5
Реализуйте функцию add_holiday_features(), которая принимает на вход две таблицы: 
* таблицу с данными о поездках;
* таблицу с данными о праздничных днях;

и возвращает обновленную таблицу с данными о поездках с добавленным в нее столбцом pickup_holiday - бинарным признаком того, начата ли поездка в праздничный день или нет (1 - да, 0 - нет). 

Чему равна медианная длительность поездки на такси в праздничные дни? Ответ приведите в секундах, округлив до целого.


In [ ]:
holiday_data = pd.read_csv('holiday_data.csv', sep=';')
holiday_data.head()

In [ ]:
holiday_data.info()

In [ ]:
holiday_data['date'] = pd.to_datetime(holiday_data['date'])

In [ ]:
def add_holiday_features(df_taxi, df_holiday):
   df_holiday['date']  = pd.to_datetime(df_holiday['date'])
   holiday_list = np.array(df_holiday['date'].dt.date)
   df_taxi['pickup_datetime'] = df_taxi['pickup_datetime'].dt.date
   df_taxi['pickup_holiday'] = df_taxi['pickup_datetime'].apply(lambda x: 1 if (x in holiday_list) else 0)
   return df_taxi 

In [ ]:
add_holiday_features(taxi_data, holiday_data)

In [ ]:
taxi_data['pickup_holiday'].value_counts()

In [ ]:
median_duration = taxi_data.loc[taxi_data['pickup_holiday'] == 1, 'trip_duration'].median()
median_duration

### Задание 2.6
Реализуйте функцию add_osrm_features(), которая принимает на вход две таблицы:
* таблицу с данными о поездках;
* таблицу с данными из OSRM;

и возвращает обновленную таблицу с данными о поездках с добавленными в нее 3 столбцами:
* total_distance;
* total_travel_time;
* number_of_steps.

а) Чему равна разница (в секундах) между медианной длительностью поездки в данных и медианной длительностью поездки, полученной из OSRM? 

В результате объединения таблиц у вас должны были получиться пропуски в столбцах с информацией из OSRM API. Это связано с тем, что для некоторых поездок не удалось выгрузить данные из веб источника. 

б) Сколько пропусков содержится в столбцах с информацией из OSRM API после объединения таблиц?

In [ ]:
osrm_data = pd.read_csv('osrm_data_train.csv')



In [ ]:
osrm_data_cut= osrm_data[['id', 'total_distance', 'total_travel_time', 'number_of_steps']] 
osrm_data_cut.head()

In [ ]:
osrm_data_cut.info()

In [ ]:
def add_osrm_features(df_taxi, df_osrm):
   df_taxi = df_taxi.merge(df_osrm, how='left', on='id')
   return df_taxi 

In [ ]:
taxi_data = add_osrm_features(taxi_data, osrm_data_cut)

In [ ]:
round(taxi_data['trip_duration'].median() - taxi_data['total_travel_time'].median())

In [ ]:
taxi_data.isnull().sum(axis=1).value_counts()

In [ ]:
def get_haversine_distance(lat1, lng1, lat2, lng2):
    # переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    # радиус земли в километрах
    EARTH_RADIUS = 6371 
    # считаем кратчайшее расстояние h по формуле Хаверсина
    lat_delta = lat2 - lat1
    lng_delta = lng2 - lng1
    d = np.sin(lat_delta * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng_delta * 0.5) ** 2
    h = 2 * EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def get_angle_direction(lat1, lng1, lat2, lng2):
    # переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    # считаем угол направления движения alpha по формуле угла пеленга
    lng_delta_rad = lng2 - lng1
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    alpha = np.degrees(np.arctan2(y, x))
    return alpha

In [ ]:
def add_geographical_features(df_taxi):
    df_taxi['haversine_distance']=df_taxi.apply(lambda x: get_haversine_distance(x['pickup_latitude'], x['pickup_longitude'], x['dropoff_latitude'], x['dropoff_longitude']), axis=1)
    df_taxi['direction']=df_taxi.apply(lambda x: get_angle_direction(x['pickup_latitude'], x['pickup_longitude'], x['dropoff_latitude'], x['dropoff_longitude']), axis=1)
    return df_taxi 



In [ ]:

taxi_data = add_geographical_features(taxi_data)

taxi_data.info()



### Задание 2.7.
Реализуйте функцию add_geographical_features(), которая принимает на вход таблицу с данными о поездках и возвращает обновленную таблицу с добавленными в нее 2 столбцами:
* haversine_distance - расстояние Хаверсина между точкой, в которой был включен счетчик, и точкой, в которой счетчик был выключен;
* direction - направление движения из точки, в которой был включен счетчик, в точку, в которой счетчик был выключен.

Чему равно медианное расстояние Хаверсина поездок (в киллометрах)? Ответ округлите до сотых.


In [ ]:
taxi_data['haversine_distance'].median()

### Задание 2.8.
Реализуйте функцию add_cluster_features(), которая принимает на вход таблицу с данными о поездках и обученный алгоритм кластеризации. Функция должна возвращать обновленную таблицу с добавленными в нее столбцом geo_cluster - географический кластер, к которому относится поездка.

Сколько поездок содержится в наименьшем по размеру географическом кластере?


In [ ]:
# создаем обучающую выборку из географических координат всех точек
coords = np.hstack((taxi_data[['pickup_latitude', 'pickup_longitude']],
                    taxi_data[['dropoff_latitude', 'dropoff_longitude']]))
# обучаем алгоритм кластеризации
kmeans = cluster.KMeans(n_clusters=10, random_state=42)
kmeans.fit(coords)

# ваш код здесь

In [ ]:
def add_claster_features(df_taxi, alg):
    df_taxi['geo_cluster'] = alg.fit_predict(coords)
    return df_taxi

In [ ]:
taxi_data = add_claster_features(taxi_data, kmeans)
taxi_data.head()

In [ ]:
taxi_data['geo_cluster'].value_counts()

### Задание 2.9.
Реализуйте функцию add_weather_features(), которая принимает на вход две таблицы:
* таблицу с данными о поездках;
* таблицу с данными о погодных условиях на каждый час;

и возвращает обновленную таблицу с данными о поездках с добавленными в нее 5 столбцами:
* temperature - температура;
* visibility - видимость;
* wind speed - средняя скорость ветра;
* precip - количество осадков;
* events - погодные явления.

а) Сколько поездок было совершено в снежную погоду?

В результате объединения у вас должны получиться записи, для которых в столбцах temperature, visibility, wind speed, precip, и events будут пропуски. Это связано с тем, что в таблице с данными о погодных условиях отсутствуют измерения для некоторых моментов времени, в которых включался счетчик поездки. 

б) Сколько процентов от общего количества наблюдений в таблице с данными о поездках занимают пропуски в столбцах с погодными условиями? Ответ приведите с точностью до сотых процента.


In [ ]:
weather_data = pd.read_csv('weather_data.csv')
weather_data_cut = weather_data[['time', 'temperature', 'visibility', 'wind speed', 'precip', 'events']]
weather_data_cut.head()

In [ ]:
weather_data_cut.isnull().sum()

In [ ]:
weather_data_cut['time'] = pd.to_datetime(weather_data_cut['time'])

In [ ]:
weather_data_cut['date'] = weather_data_cut['time'].dt.date
weather_data_cut['hour'] = weather_data_cut['time'].dt.hour

In [ ]:
weather_data_cut.drop('time', axis=1)

In [ ]:
def add_weather_features(taxi_data, weather_data_cut):

    taxi_data = pd.merge(taxi_data, weather_data_cut, how='left', left_on=['pickup_date', 'pickup_hour'], right_on=['date', 'hour'])
    taxi_data.drop(['date', 'hour'], axis=1, inplace=True)
    
    return taxi_data  


    

In [ ]:

taxi_data = add_weather_features(taxi_data, weather_data_cut)
taxi_data.info()

In [ ]:
taxi_data['events'].value_counts()

In [ ]:
taxi_data.isnull().mean()*100

### Задание 2.10.
Реализуйте функцию fill_null_weather_data(), которая принимает на вход которая принимает на вход таблицу с данными о поездках. Функция должна заполнять пропущенные значения в столбцах.

Пропуски в столбцах с погодными условиями -  temperature, visibility, wind speed, precip заполните медианным значением температуры, влажности, скорости ветра и видимости в зависимости от даты начала поездки. Для этого сгруппируйте данные по столбцу pickup_date и рассчитайте медиану в каждой группе, после чего с помощью комбинации методов transform() и fillna() заполните пропуски. 
Пропуски в столбце events заполните строкой 'None' - символом отсутствия погодных явлений (снега/дождя/тумана). 

Пропуски в столбцах с информацией из OSRM API - total_distance, total_travel_time и number_of_steps заполните медианным значением по столбцам. 

Чему равна медиана в столбце temperature после заполнения пропусков? Ответ округлите до десятых.


In [ ]:
def fill_null_weather_data(df_taxi):
    col = ['temperature', 'visibility', 'wind speed', 'precip']
    col2 = ['total_distance', 'total_travel_time', 'number_of_steps']
    for i in col:
        df_taxi[i] = df_taxi[i].fillna(df_taxi.groupby('pickup_date')[i].transform('median'))
    for j in col2:
        # df_taxi = df_taxi.fillna(i:df_taxi[j].median())
        df_taxi[j] = df_taxi[j].fillna(taxi_data[j].median())
    #df_taxi['events'] = df_taxi['events'].replace(np.nan, 'None')
    df_taxi['events'] = df_taxi['events'].fillna('None') 
    return df_taxi
    

In [ ]:
taxi_data = fill_null_weather_data(taxi_data)
taxi_data.info()

In [ ]:
taxi_data['temperature'].median()

В завершение первой части найдем очевидные выбросы в целевой переменной - длительности поездки. 

Проще всего найти слишком продолжительные поездки. Давайте условимся, что выбросами будут считаться поездки, длительность которых превышает 24 часа. 

Чуть сложнее с анализом поездок, длительность которых слишком мала. Потому что к ним относятся действительно реальные поездки на короткие расстояния, поездки, которые были отменены через секунду после того как включился счетчик, а также “телепортации” - перемещение на большие расстояния за считанные секунды. 
Условимся, что мы будем считать выбросами только последнюю группу. Как же нам их обнаружить наиболее простым способом?

Можно воспользоваться информацией о кратчайшем расстоянии, которое проезжает такси. Вычислить среднюю скорость автомобиля на кратчайшем пути следующим образом: 
$$avg\_speed= \frac{total\_distance}{1000*trip\_duration}*3600$$
Если мы построим диаграмму рассеяния средней скорости движения автомобилей, мы увидим следующую картину:


In [ ]:
avg_speed = taxi_data['total_distance'] / taxi_data['trip_duration'] * 3.6
fig, ax = plt.subplots(figsize=(10, 5))
sns.scatterplot(x=avg_speed.index, y=avg_speed, ax=ax)
ax.set_xlabel('Index')
ax.set_ylabel('Average speed');

Как раз отсюда мы видим, что у нас есть “поездки-телепортации”, для которых средняя скорость более 1000 км/ч. Даже есть такая, средняя скорость которой составляла более 12000 км/ч! 

Давайте условимся, что предельная средняя скорость, которую могут развивать таксисты будет 300 км/ч. 


### Задание 2.11.
Найдите поездки, длительность которых превышает 24 часа. И удалите их из набора данных.

а) Сколько выбросов по признаку длительности поездки вам удалось найти?

Найдите поездки, средняя скорость которых по кратчайшему пути превышает 300 км/ч и удалите их из данных. 

б) Сколько выбросов по признаку скорости вам удалось найти?

In [ ]:
duration_drop = taxi_data[taxi_data['trip_duration'] > (24*60*60)].index
len(duration_drop)

In [ ]:
taxi_data['avg_speed'] = avg_speed
speed_drop = taxi_data[taxi_data['avg_speed'] > 300].index
len(speed_drop)


In [ ]:
total_drop = duration_drop.append(speed_drop)
taxi_data = taxi_data.drop(index=total_drop)

## 3. Разведывательный анализ данных (EDA)

В этой части нашего проекта мы с вами:
* Исследуем сформированный набор данных; 
* Попробуем найти закономерности, позволяющие сформулировать предварительные гипотезы относительно того, какие факторы являются решающими в определении длительности поездки;
* Дополним наш анализ визуализациями, иллюстрирующими; исследование. Постарайтесь оформлять диаграммы с душой, а не «для галочки»: навыки визуализации полученных выводов обязательно пригодятся вам в будущем.


Начинаем с целевого признака. Забегая вперед, скажем, что основной метрикой качества решения поставленной задачи будет RMSLE - Root Mean Squared Log Error, которая вычисляется на основе целевой переменной в логарифмическом масштабе. В таком случае целесообразно сразу логарифмировать признак длительности поездки и рассматривать при анализе логарифм в качестве целевого признака:
$$trip\_duration\_log = log(trip\_duration+1),$$
где под символом log подразумевается натуральный логарифм.


In [ ]:
taxi_data['trip_duration_log'] = np.log(taxi_data['trip_duration']+1)

### Задание 3.1.
Постройте гистограмму и коробчатую диаграмму длительности поездок в логарифмическом масштабе (trip_duration_log). 
Исходя из визуализации, сделайте предположение, является ли полученное распределение нормальным? 
Проверьте свою гипотезу с помощью теста Д’Агостино при уровне значимости $\alpha=0.05$. 

а) Чему равен вычисленный p-value? Ответ округлите до сотых.

б) Является ли распределение длительности поездок в логарифмическом масштабе нормальным?

In [ ]:
plt.hist(taxi_data.trip_duration_log)
plt.title('Распределение признака длительность поездки');

In [ ]:
plt.boxplot(taxi_data.trip_duration_log)
plt.title('Распределение признака длительность поездки');

In [ ]:
from scipy.stats import normaltest

# Проверка нормальности распределения с помощью теста Д'Агостино-Пирсона
statistic, p_value = normaltest(taxi_data['trip_duration_log'])

alpha = 0.05  # Уровень значимости

if p_value > alpha:
    print("Отвергаем гипотезу о нормальности распределения")
else:
    print("Не получилось отвергнуть гипотезу о нормальности распределения")

In [ ]:
p_value

### Задание 3.2.
Постройте визуализацию, которая позволит сравнить распределение длительности поездки в логарифмическом масштабе (trip_duration_log) в зависимости от таксопарка (vendor_id). 

Сравните два распределения между собой.

In [ ]:
taxi_data['vendor_id'].value_counts()

In [ ]:
# Разделение данных по таксопаркам
vendor_A_data = taxi_data.loc[taxi_data['vendor_id'] == 1, 'trip_duration_log']
vendor_B_data = taxi_data.loc[taxi_data['vendor_id'] == 2, 'trip_duration_log']

# Построение гистограммы
plt.hist([vendor_A_data, vendor_B_data], bins=10, label=['Vendor A', 'Vendor B'], log=True)

# Настройка осей и заголовка
plt.xlabel('Log Trip Duration')
plt.ylabel('Frequency')
plt.title('Histogram of Log Trip Duration by Vendor')

# Добавление легенды
plt.legend()

# Отображение гистограммы
plt.show()

# Построение коробчатой диаграммы
plt.boxplot([vendor_A_data, vendor_B_data], labels=['Vendor A', 'Vendor B'], vert=False)

# Настройка осей и заголовка
plt.xlabel('Log Trip Duration')
plt.title('Boxplot of Log Trip Duration by Vendor')

# Отображение коробчатой диаграммы
plt.show()

### Задание 3.3.
Постройте визуализацию, которая позволит сравнить распределение длительности поездки в логарифмическом масштабе (trip_duration_log) в зависимости от признака отправки сообщения поставщику (store_and_fwd_flag). 

Сравните два распределения между собой.

In [ ]:
taxi_data['store_and_fwd_flag'].value_counts()

In [ ]:
# Разделение данных по таксопаркам
flag_n = taxi_data.loc[taxi_data['store_and_fwd_flag'] == 'N', 'trip_duration_log']
flag_y = taxi_data.loc[taxi_data['store_and_fwd_flag'] == 'Y', 'trip_duration_log']

# Построение гистограммы
plt.hist([flag_n, flag_y], bins=10, label=['Flag N', 'Flag Y'], log=True)

# Настройка осей и заголовка
plt.xlabel('Log Trip Duration')
plt.ylabel('Frequency')
plt.title('Histogram of Log Trip Duration by Flag')

# Добавление легенды
plt.legend()

# Отображение гистограммы
plt.show()

# Построение коробчатой диаграммы
plt.boxplot([flag_n, flag_y], labels=['Flag N', 'Flag Y'], vert=False)

# Настройка осей и заголовка
plt.xlabel('Log Trip Duration')
plt.title('Boxplot of Log Trip Duration by Flag')

# Отображение коробчатой диаграммы
plt.show()

### Задание 3.4.
Постройте две визуализации:
* Распределение количества поездок в зависимости от часа дня;
* Зависимость медианной длительности поездки от часа дня.

На основе построенных графиков ответьте на следующие вопросы:

а) В какое время суток такси заказывают реже всего?

б) В какое время суток наблюдается пик медианной длительности поездок?

In [ ]:
taxi_data['pickup_hour'].value_counts()

In [ ]:
def time_group(x):
    if 0 <= x <= 5:
        return '0-5'
    elif 6 <= x <= 12:
        return '6-12'
    elif 13 <= x <= 18:
        return '13-18'
    elif 18 <= x <= 23:
        return '18-23' 
  


taxi_data['time_group'] = taxi_data['pickup_hour'].apply(time_group)
taxi_data['time_group'].value_counts()

In [ ]:
plt.hist(taxi_data.time_group)
plt.title('Распределение временных промежутков');

In [ ]:
taxi_data.groupby('time_group')['trip_duration'].median()

### Задание 3.5.
Постройте две визуализации:
* Распределение количества поездок в зависимости от дня недели;
*  Зависимость медианной длительности поездки от дня недели.

На основе построенных графиков ответьте на следующие вопросы:
а) В какой день недели совершается больше всего поездок?
б) В какой день недели медианная длительность поездок наименьшая?


In [ ]:
taxi_data['pickup_day_of_week'].value_counts()

In [ ]:
plt.hist(taxi_data.pickup_day_of_week)
plt.title('Распределение по дням недели');

In [ ]:
taxi_data.groupby('pickup_day_of_week')['trip_duration'].median()

### Задание 3.6.
Посмотрим на обе временные характеристики одновременно. 

Постройте сводную таблицу, по строкам которой отложены часы (pickup_hour), по столбцам - дни недели (pickup_day_of_week), а в ячейках - медианная длительность поездки (trip_duration). 

Визуализируйте полученную сводную таблицу с помощью тепловой карты (рекомендуемая палитра - coolwarm).

In [ ]:
# Создание сводной таблицы
pivot_table = pd.pivot_table(taxi_data, values='trip_duration', index='pickup_hour', columns='pickup_day_of_week', aggfunc='median')

pivot_table

In [ ]:
# Визуализация с помощью тепловой карты
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt=".0f")

# Настройка осей и заголовка
plt.xlabel('Day of Week')
plt.ylabel('Hour of Day')
plt.title('Median Trip Duration by Hour and Day')

# Отображение тепловой карты
plt.show()

### Задание 3.7.
Постройте две диаграммы рассеяния (scatter-диаграммы):
* первая должна иллюстрировать географическое расположение точек начала поездок (pickup_longitude, pickup_latitude) 
* вторая должна географическое расположение точек завершения поездок (dropoff_longitude, dropoff_latitude).

Для этого на диаграммах по оси абсцисс отложите широту (longitude), а по оси ординат - долготу (latitude). 
Включите в визуализацию только те точки, которые находятся в пределах Нью-Йорка - добавьте следующие ограничения на границы осей абсцисс и ординат:
 
city_long_border = (-74.03, -73.75)

city_lat_border = (40.63, 40.85)

Добавьте на диаграммы расцветку по десяти географическим кластерам (geo_cluster), которые мы сгенерировали ранее. 

**Рекомендация:** для наглядности уменьшите размер точек на диаграмме рассеяния.  


In [ ]:
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)


In [ ]:
# Ограничения на границы осей абсцисс и ординат
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)


# Построение scatter-диаграммы для точек начала поездок
plt.scatter(taxi_data['pickup_longitude'], taxi_data['pickup_latitude'], c=taxi_data['geo_cluster'], cmap='tab10')
plt.xlim(city_long_border)
plt.ylim(city_lat_border)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Pickup Locations')
plt.colorbar(label='Geo Cluster')
plt.show()

# Построение scatter-диаграммы для точек завершения поездок
plt.scatter(taxi_data['dropoff_longitude'], taxi_data['dropoff_latitude'], c=taxi_data['geo_cluster'], cmap='tab10')
plt.xlim(city_long_border)
plt.ylim(city_lat_border)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Dropoff Locations')
plt.colorbar(label='Geo Cluster')
plt.show()

In [ ]:


city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)

df_start = taxi_data[
      (taxi_data['pickup_longitude'] >= -74.03) &
      (taxi_data['pickup_longitude'] <= -73.75)&
      (taxi_data['pickup_latitude'] >= 40.63) &
      (taxi_data['pickup_latitude'] <= 40.85)]

df_end = taxi_data[
      (taxi_data['dropoff_longitude'] >= -74.03) &
      (taxi_data['dropoff_longitude'] <= -73.75) &
      (taxi_data['dropoff_latitude'] >= 40.63) &
      (taxi_data['dropoff_latitude'] <= 40.85)]

print(df_start['geo_cluster'].unique())
print(df_end['geo_cluster'].unique())



In [ ]:
plt.figure(figsize=(20, 20))
geo_start = sns.scatterplot(data=df_start, x='pickup_longitude', y='pickup_latitude', hue=df_start['geo_cluster'], s=2, palette='colorblind')
plt.title('Географическое расположение точек начала поезки с разбивкой по Гео-кластерам', fontsize=16)
geo_start.set_facecolor("white")

In [ ]:


plt.figure(figsize=(20, 20))
geo_end = sns.scatterplot(data=df_end, x='dropoff_longitude', y='dropoff_latitude', hue=df_end['geo_cluster'], s=2, palette='colorblind')
plt.title('Географическое расположение точек окончания поезки с разбивкой по Гео-кластерам', fontsize=16)
geo_end.set_facecolor("white")



## 4. Отбор и преобразование признаков

Перед тем как перейти к построению модели, осталось сделать ещё несколько шагов.
* Следует помнить, что многие алгоритмы машинного обучения не могут обрабатывать категориальные признаки в их обычном виде. Поэтому нам необходимо их закодировать;
* Надо отобрать признаки, которые мы будем использовать для обучения модели;
*  Необходимо масштабировать и трансформировать некоторые признаки для того, чтобы улучшить сходимость моделей, в основе которых лежат численные методы.


In [ ]:
print('Shape of data: {}'.format(taxi_data.shape))
print('Columns: {}'.format(taxi_data.columns))

Для удобства работы сделаем копию исходной таблицы с поездками:

In [ ]:
train_data = taxi_data[['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'val_time', 'pickup_date', 'pickup_hour',
       'pickup_day_of_week', 'pickup_holiday', 'total_distance',
       'total_travel_time', 'number_of_steps', 'haversine_distance',
       'direction', 'geo_cluster', 'time', 'temperature', 'visibility',
       'wind speed', 'precip', 'events', 'avg_speed', 'trip_duration_log',
       'time_group']]
train_data.head()

### Задание 4.1.
Сразу позаботимся об очевидных неинформативных и избыточных признаках. 

а) Какой из признаков является уникальным для каждой поездки и не несет полезной информации в определении ее продолжительности?

б) Утечка данных (data leak) - это…

в) Подумайте, наличие какого из признаков в обучающем наборе данных создает утечку данных?

г) Исключите выбранные в пунктах а) и в) признаки из исходной таблицы с данными. Сколько столбцов в таблице у вас осталось?


In [ ]:
train_data = train_data.drop(['val_time', 'id', 'dropoff_datetime', 'time_group'], axis=1)

In [ ]:
train_data = train_data.drop('avg_speed', axis=1)

Ранее мы извлекли всю необходимую для нас информацию из даты начала поездки, теперь мы можем избавиться от этих признаков, так как они нам больше не понадобятся:


In [ ]:
drop_columns = ['pickup_datetime', 'pickup_date']
train_data = train_data.drop(drop_columns, axis=1)
print('Shape of data:  {}'.format(train_data.shape))

In [ ]:
train_data = train_data.drop('time', axis=1)

In [ ]:
print('Shape of data:  {}'.format(train_data.shape))

### Задание 4.2.

Закодируйте признак vendor_id в таблице train_data таким образом, чтобы он был равен 0, если идентификатор таксопарка равен 1, и 1 — в противном случае.

Закодируйте признак store_and_fwd_flag в таблице train_data таким образом, чтобы он был равен 0, если флаг выставлен в значение 'N', и 1 — в противном случае.

а) Рассчитайте среднее по закодированному столбцу vendor_id. Ответ приведите с точностью до сотых.

б) Рассчитайте среднее по закодированному столбцу store_and_fwd_flag. Ответ приведите с точностью до тысячных.



In [ ]:
def vendor(x):
    if x == 1:
        return 0
    else:
        return 1

train_data['vendor_id'] = train_data['vendor_id'].apply(vendor)
train_data['vendor_id'].value_counts()

In [ ]:
def flag(x):
    if x == 'N':
        return 0
    else:
        return 1

train_data['store_and_fwd_flag'] = train_data['store_and_fwd_flag'].apply(flag)
train_data['store_and_fwd_flag'].value_counts()

In [ ]:


print(train_data['vendor_id'].mean())
print(train_data['store_and_fwd_flag'].mean())



### Задание 4.3.
Создайте таблицу data_onehot из закодированных однократным кодированием признаков pickup_day_of_week, geo_cluster и events в таблице train_data с помощью OneHotEncoder из библиотеки sklearn. Параметр drop выставите в значение 'first', чтобы удалять первый бинарный столбец, тем самым не создавая излишних признаков.

В результате работы OneHotEncoder вы получите безымянный numpy-массив, который нам будет необходимо преобразовать обратно в DataFrame, для более удобной работы в дальнейшем. Чтобы получить имена закодированных столбцов у объекта типа OneHotEncoder есть специальный метод get_feature_names_out(). Он возвращает список новых закодированных имен столбцов в формате <оригинальное имя столбца>_<имя категории>.

Пример использования:

``` python
# Получаем закодированные имена столбцов
column_names = one_hot_encoder.get_feature_names_out()
# Составляем DataFrame из закодированных признаков
data_onehot = pd.DataFrame(data_onehot, columns=column_names)
```

В этом псевдокоде:
* one_hot_encoder - объект класса OneHotEncoder
* data_onehot - numpy-массив, полученный в результате трансформации кодировщиком

В результате выполнения задания у вас должен быть образован DataFrame `data_onehot`, который содержит кодированные категориальные признаки pickup_day_of_week, geo_cluster и events. 


Сколько бинарных столбцов у вас получилось сгенерировать с помощью однократного кодирования?


In [ ]:
columns_to_change = ['pickup_day_of_week', 'geo_cluster', 'events']

In [ ]:
import category_encoders as ce # импорт для работы с кодировщиком

one_hot_encoder = preprocessing.OneHotEncoder(drop='first')
data_onehot = one_hot_encoder.fit_transform(train_data[columns_to_change]).toarray()
# Получаем закодированные имена столбцов
column_names = one_hot_encoder.get_feature_names_out()
# Составляем DataFrame из закодированных признаков
data_onehot = pd.DataFrame(data_onehot, columns=column_names)


In [ ]:
data_onehot.shape[1]

Добавим полученную таблицу с закодированными признаками:

In [ ]:
train_data = pd.concat(
    [train_data.reset_index(drop=True).drop(columns_to_change, axis=1), data_onehot], 
    axis=1
)
print('Shape of data: {}'.format(train_data.shape))

Теперь, когда категориальные признаки предобработаны, сформируем матрицу наблюдений X, вектор целевой переменной y и его логарифм y_log. В матрицу наблюдений войдут все столбцы из таблицы с поездками за исключением целевого признака trip_duration и его логарифмированной версии trip_duration_log:


In [ ]:
X = train_data.drop(['trip_duration', 'trip_duration_log'], axis=1)
y = train_data['trip_duration']
y_log = train_data['trip_duration_log']

Все наши модели мы будем обучать на логарифмированной версии y_log. 

Выбранный тип валидации - hold-out. Разобьем выборку на обучающую и валидационную в соотношении 67/33:

In [ ]:
X_train, X_valid, y_train_log, y_valid_log = model_selection.train_test_split(
    X, y_log, 
    test_size=0.33, 
    random_state=42
)

На данный момент у нас достаточно много признаков: скорее всего, не все из них будут важны. Давайте оставим лишь те, которые сильнее всего связаны с целевой переменной и точно будут вносить вклад в повышение качества модели.


### Задание 4.4.
С помощью SelectKBest отберите 25 признаков, наилучшим образом подходящих для предсказания целевой переменной в логарифмическом масштабе. Отбор реализуйте по обучающей выборке, используя параметр score_func = f_regression.

Укажите признаки, которые вошли в список отобранных


In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

selector_skb = SelectKBest(f_regression, k=25)
selector_skb.fit(X_train, y_train_log)
 
selector_columns = selector_skb.get_feature_names_out()
X_train = pd.DataFrame(X_train, columns=selector_columns)
X_valid = pd.DataFrame(X_valid, columns=selector_columns)


In [ ]:
X_train.shape

In [ ]:
X_train = pd.DataFrame(X_train, columns=selector_columns)
X_train.info()

In [ ]:
X_valid.shape

In [ ]:
X_valid = pd.DataFrame(X_valid, columns=selector_columns)
X_valid.info()

In [ ]:
selector_columns

Так как мы будем использовать различные модели, в том числе внутри которых заложены численные методы оптимизации, то давайте заранее позаботимся о масштабировании факторов. 


### Задание 4.5.
Нормализуйте предикторы в обучающей и валидационной выборках с помощью MinMaxScaler из библиотеки sklearn. Помните, что обучение нормализатора производится на обучающей выборке, а трансформация на обучающей и валидационной!

Рассчитайте среднее арифметическое для первого предиктора (т. е. для первого столбца матрицы) из валидационной выборки. Ответ округлите до сотых.


In [ ]:
mm_scaler = preprocessing.MinMaxScaler()

In [ ]:
mm_scaler.fit(X_train)
X_train = mm_scaler.transform(X_train)
X_valid = mm_scaler.transform(X_valid)

X_valid[:, 0].mean()

## 5. Решение задачи регрессии: линейная регрессия и деревья решений

Определим метрику, по которой мы будем измерять качество наших моделей. Мы будем следовать канонам исходного соревнования на Kaggle и в качестве метрики использовать RMSLE (Root Mean Squared Log Error), которая вычисляется как:
$$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n(log(y_i+1)-log(\hat{y_i}+1))^2},$$
где:
* $y_i$ - истинная длительность i-ой поездки на такси (trip_duration)
* $\hat{y_i}$- предсказанная моделью длительность i-ой поездки на такси

Заметим, что логарифмирование целевого признака мы уже провели заранее, поэтому нам будет достаточно вычислить метрику RMSE для модели, обученной прогнозировать длительность поездки такси в логарифмическом масштабе:
$$z_i=log(y_i+1),$$
$$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n(z_i-\hat{z_i})^2}=\sqrt{MSE(z_i,\hat{z_i})}$$ 



### Задание 5.1.
Постройте модель линейной регрессии на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Все параметры оставьте по умолчанию.

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [ ]:
from sklearn.linear_model import LinearRegression
# Создаем объект модели линейной регрессии
model = LinearRegression()

In [ ]:
# Обучаем модель 
model.fit(X_train, y_train_log)

In [ ]:
y_train_predict = model.predict(X_train)
y_valid_predict = model.predict(X_valid)

In [ ]:
def rmsle (y, y_pred):
    # rmsl_error = np.sqrt(metrics.mean_squared_error(y, y_pred))
    rmsl_error = metrics.mean_squared_error(y, y_pred, squared=False)
    return rmsl_error

In [ ]:
print(rmsle(y_train_log, y_train_predict))
print(rmsle(y_valid_log, y_valid_predict))

### Задание 5.2.
Сгенерируйте полиномиальные признаки 2-ой степени с помощью PolynomialFeatures из библиотеки sklearn. Параметр include_bias выставите в значение False.

Постройте модель полиномиальной регрессии 2-ой степени на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Все параметры оставьте по умолчанию.

а) Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.

б) Наблюдаются ли у вашей модели признаки переобучения?


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
poly.fit(X_train)
#Производим преобразование для каждой из выборок
X_train_poly = poly.transform(X_train)
X_valid_poly = poly.transform(X_valid)

#Создаём объект класса LinearRegression
#lr_model_poly = linear_model.LinearRegression()
#Обучаем модель по МНК
#lr_model_poly.fit(X_train_poly, y_train_log)
#Делаем предсказание для тренировочной выборки
#y_train_predict_poly = lr_model_poly.predict(X_train_poly)
#Делаем предсказание для тестовой выборки
#y_valid_predict_poly = lr_model_poly.predict(X_valid_poly)
 
#Рассчитываем метрики для двух выборок
#print(rmsle(y_train_log, y_train_predict_poly))
#print(rmsle(y_valid_log, y_valid_predict_poly))

### Задание 5.3.
Постройте модель полиномиальной регрессии 2-ой степени с L2-регуляризацией (регуляризация по Тихонову) на обучающей выборке  (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Коэффициент регуляризации $\alpha$ установите равным 1, остальные параметры оставьте по умолчанию.

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [ ]:
#Инициализируем объект класса линейная регрессия с L2-регуляризацией 
ridge_lr_poly = linear_model.Ridge(alpha=1)
#Обучаем модель предсказывать логарифм целевого признака
ridge_lr_poly.fit(X_train_poly, y_train_log)
#Делаем предсказание для каждой из выборок
#Если бы обучили на логарифме, то от результата необходимо было бы взять обратную функцию - экспоненту
y_train_pred = ridge_lr_poly.predict(X_train_poly)
y_valid_pred = ridge_lr_poly.predict(X_valid_poly)
#Выводим результирующие метрики
print(rmsle(y_train_log, y_train_pred))
print(rmsle(y_valid_log, y_valid_pred))

### Задание 5.4.
Постройте модель дерева решений (DecisionTreeRegressor) на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Все параметры оставьте по умолчанию. 

а) Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.

б) Наблюдаются ли у вашей модели признаки переобучения?


In [ ]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()

In [ ]:
clf = clf.fit(X_train, y_train_log)

In [ ]:
y_train_predict_clf = clf.predict(X_train)
y_valid_predict_clf = clf.predict(X_valid)

In [ ]:
print(rmsle(y_train_log, y_train_predict_clf))
print(rmsle(y_valid_log, y_valid_predict_clf))

### Задание 5.5.
Переберите все возможные варианты глубины дерева решений в диапазоне от 7 до 20:

max_depths = range(7, 20)

Параметр random_state задайте равным 42.

Постройте линейные графики изменения метрики RMSE на тренировочной и валидационной выборках в зависимости от значения параметра глубины дерева решений. 

а) Найдите оптимальное значение максимальной глубины дерева, для которой будет наблюдаться минимальное значение RMSLE на обучающей выборке, но при этом еще не будет наблюдаться переобучение (валидационная кривая еще не начинает возрастать).

б) Чему равно значение метрик RMSLE на тренировочной и валидационной выборках для дерева решений с выбранной оптимальной глубиной? Ответ округлите до сотых.


In [ ]:

max_depths = range(7, 20)
#Создаем пустые списки, в которые будем добавлять результаты 
train_scores = []
valid_scores = []
for max_depth in max_depths:
    #Создаем объект класса дерево решений
    regr = tree.DecisionTreeRegressor(max_depth=max_depth, random_state=42)
    #Обучаем модель предсказывать логарифм целевого признака
    regr.fit(X_train, y_train_log)
    #Делаем предсказание для каждой из выборок
    y_train_pred = regr.predict(X_train)
    y_valid_pred = regr.predict(X_valid)
    #Рассчитываем метрику для двух выборок и добавляем их в списки
    train_scores.append(rmsle(y_train_log, y_train_pred))
    valid_scores.append(rmsle(y_valid_log, y_valid_pred))
 
#Визуализируем изменение RMSLE в зависимости от max_depth
fig, ax = plt.subplots(figsize=(12, 4)) #фигура + координатная плоскость
ax.plot(max_depths, train_scores, label='Train') #линейный график для тренировочной выборки
ax.plot(max_depths, valid_scores, label='Valid') #линейный график для валидационной выборки
ax.set_xlabel('max_depth') #название оси абсцисс
ax.set_ylabel('RMSLE') #название оси ординат
ax.set_xticks(max_depths) #метки по оси абцисс
ax.xaxis.set_tick_params(rotation=45) #поворот меток на оси абсцисс
ax.legend(); #отображение легенды

In [ ]:


regr = tree.DecisionTreeRegressor(max_depth=12,random_state=42)
regr.fit(X_train, y_train_log)
##DecisionTreeRegressor(max_depth=10, random_state=42)
y_train_pred = regr.predict(X_train)
y_valid_pred = regr.predict(X_valid)
print(rmsle(y_train_log, y_train_pred))
print(rmsle(y_valid_log, y_valid_pred))



## 6. Решение задачи регрессии: ансамблевые методы и построение прогноза

Переходим к тяжелой артиллерии: ансамблевым алгоритмам. 

### Задание 6.1.

Постройте модель случайного леса на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). В качестве гиперпараметров укажите следующие:
* n_estimators=200,
* max_depth=12,
* criterion='squared_error',
* min_samples_split=20,
* random_state=42

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(
    n_estimators = 200, 
    max_depth=12, #максимальная глубина дерева
    criterion='squared_error',
    min_samples_split=20,
    random_state=42 #датчик генератора случайных чисел
)
# Обучаем модель
rfr.fit(X_train, y_train_log)

y_train_pred = rfr.predict(X_train)
y_valid_pred = rfr.predict(X_valid)
print(rmsle(y_train_log, y_train_pred))
print(rmsle(y_valid_log, y_valid_pred))


### Задание 6.2.
Постройте модель градиентного бустинга над деревьями решений (GradientBoostingRegressor) на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). В качестве гиперпараметров укажите следующие:
* learning_rate=0.5,
* n_estimators=100,
* max_depth=6, 
* min_samples_split=30,
* random_state=42

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Создаем объект класса градиентный бустинг
gb = GradientBoostingRegressor(
    learning_rate=0.5,
    max_depth=6, #максимальная глубина дерева
    n_estimators=100, #количество деревьев в ансамбле
    min_samples_split=30,
    random_state=42 #датчик генератора случайных чисел
)

# Обучаем модель
gb.fit(X_train, y_train_log)

# Формируем предсказание для тестовой выборки
y_train_pred = gb.predict(X_train)
y_valid_pred = gb.predict(X_valid)
print(rmsle(y_train_log, y_train_pred))
print(rmsle(y_valid_log, y_valid_pred))

### Задание 6.3.
Какая из построенных вами моделей показала наилучший результат (наименьшее значение RMSLE на валидационной выборке)?
* Линейная регрессия
* Полиномиальная регрессия 2ой степени
* Дерево решений
* Случайный лес
* Градиентный бустинг над деревьями решений


### Задание 6.4.
Постройте столбчатую диаграмму коэффициентов значимости каждого из факторов.

Укажите топ-3 наиболее значимых для предсказания целевого признака - длительности поездки в логарифмическом масштабе - факторов.


In [ ]:
y = y_train_log
X = pd.DataFrame(pd.DataFrame(X_train, columns=selector_columns))

# Get feature importances from the trained model
importances = gb.feature_importances_

# Get feature names
feature_names = X.columns

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Select top 3 features
top_features = feature_names[indices][:3]
top_importances = importances[indices][:3]

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(range(len(top_features)), top_importances, align='center')
plt.xticks(range(len(top_features)), top_features, rotation=45)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Top 3 Feature Importances')
plt.show()

### Задание 6.5.
Для лучшей из построенных моделей рассчитайте медианную абсолютную ошибку (MeAE - в sklearn функция median_absolute_error) предсказания длительности поездки такси на валидационной выборке:
$$ MeAE = median(|y_i-\hat{y_i}|)$$

Значение метрики MeAE переведите в минуты и округлите до десятых.


In [ ]:
from sklearn.metrics import median_absolute_error
y = np.exp(y_valid_log)-1
y_pred = np.exp(gb.predict(X_valid))-1
#Рассчитываем MeAE
#print('MeAE score: {:.1f}'.format(metrics.median_absolute_error(y, y_valid_pred))/60)
print('MeAE score:', (metrics.median_absolute_error(y, y_pred))/60)

In [ ]:


data = {
    'Наименование модели': ['Линейная регрессия',
                            'Линейная регрессия с полиноминальными признаками второго порядка',
                            'Модель Ridge с L2 регуляризацией',
                            'Дерево решений по умолчанию',
                            'Дерево решений с параметрами',
                            'Случайный лес',
                            'Градиентный бустинг над деревьями решений'],
    'Train rmsle': [0.54, 0.47, 0.48, 0, 0.41, 0.4, 0.37],
    'Valid rmsle': [0.54, 0.7, 0.48, 0.57, 0.43, 0.41, 0.39]
}

df = pd.DataFrame(data)

df



Финальный шаг - сделать submit -  предсказание для отложенного тестового набора данных. 

Прочитаем тестовые данные и заранее выделим столбец с идентификаторами поездок из тестового набора данных. Он нам еще пригодится:


In [ ]:
test_data = pd.read_csv("Project5_test_data.csv")
osrm_data_test = pd.read_csv("Project5_osrm_data_test.csv")
test_id = test_data['id']

Перед созданием прогноза для тестовой выборки необходимо произвести все манипуляции с данными, которые мы производили с тренировочной выборкой, а именно:
* Перевести признак pickup_datetime в формат datetime;
* Добавить новые признаки (временные, географические, погодные и другие факторы);
* Произвести очистку данных от пропусков;
* Произвести кодировку категориальных признаков:
    * Закодировать бинарные признаки;
    * Закодировать номинальные признаки с помощью обученного на тренировочной выборке OneHotEncoder’а;
* Сформировать матрицу наблюдений, оставив в таблице только те признаки, которые были отобраны с помощью SelectKBest;
* Нормализовать данные с помощью обученного на тренировочной выборке MinMaxScaler’а.


In [ ]:
test_data['pickup_datetime']=pd.to_datetime(test_data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
test_data = add_datetime_features(test_data)
test_data = add_holiday_features(test_data, holiday_data)
test_data = add_osrm_features(test_data, osrm_data_test)
test_data = add_geographical_features(test_data)
test_data = add_claster_features(test_data, kmeans)
test_data = add_weather_features(test_data, weather_data)
test_data = fill_null_weather_data (test_data)

test_data['vendor_id'] = test_data['vendor_id'].apply(lambda x: 0 if x == 1 else 1)
test_data['store_and_fwd_flag'] = test_data['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)
test_data_onehot = one_hot_encoder.fit_transform(test_data[columns_to_change]).toarray()
# test_data_onehot = one_hot_encoder.fit(test_data[columns_to_change]).toarray() # предложение ментора
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
test_data_onehot = pd.DataFrame(test_data_onehot, columns=column_names)

test_data = pd.concat(
    [test_data.reset_index(drop=True).drop(columns_to_change, axis=1), test_data_onehot], 
    axis=1
)
X_test = test_data[selector_columns]
X_test_scaled = mm_scaler.transform(X_test)



Только после выполнения всех этих шагов можно сделать предсказание длительности поездки для тестовой выборки. Не забудьте перевести предсказания из логарифмического масштаба в истинный, используя формулу:
$$y_i=exp(z_i)-1$$

После того, как вы сформируете предсказание длительности поездок на тестовой выборке вам необходимо будет создать submission-файл в формате csv, отправить его на платформу Kaggle и посмотреть на результирующее значение метрики RMSLE на тестовой выборке.

Код для создания submission-файла:


In [ ]:
# ваш код здесь
submission = pd.DataFrame({'id': test_id, 'trip_duration': y_test_predict})
submission.to_csv('data/submission_gb.csv', index=False)

### **В качестве бонуса**

В завершение по ансамблевым мы предлагаем вам попробовать улучшить свое предсказание, воспользовавшись моделью экстремального градиентного бустинга (XGBoost) из библиотеки xgboost.

**XGBoost** - современная модель машинного обучения, которая является продолжением идеи градиентного бустинга Фридмана. У нее есть несколько преимуществ по сравнению с классической моделью градиентного бустинга из библиотеки sklearn: повышенная производительность путем параллелизации процесса обучения, повышенное качество решения за счет усовершенствования алгоритма бустинга, меньшая склонность к переобучению и широкий функционал возможности управления параметрами модели.


Для ее использования необходимо для начала установить пакет xgboost:

In [ ]:
#!pip install xgboost

После чего модуль можно импортировать:

In [ ]:
import xgboost as xgb

Перед обучением модели необходимо перевести наборы данных в тип данных xgboost.DMatrix:

In [ ]:
# Создание матриц наблюдений в формате DMatrix
dtrain = xgb.DMatrix(X_train_scaled, label=y_train_log, feature_names=best_features)
dvalid = xgb.DMatrix(X_valid_scaled, label=y_valid_log, feature_names=best_features)
dtest = xgb.DMatrix(X_test_scaled, feature_names=best_features)

Обучение модели XGBoost происходит с помощью метода train, в который необходимо передать параметры модели, набор данных, количество базовых моделей в ансамбле, а также дополнительные параметры:


In [ ]:
# Гиперпараметры модели
xgb_pars = {'min_child_weight': 20, 'eta': 0.1, 'colsample_bytree': 0.9, 
            'max_depth': 6, 'subsample': 0.9, 'lambda': 1, 'nthread': -1, 
            'booster' : 'gbtree', 'eval_metric': 'rmse', 'objective': 'reg:squarederror'
           }
# Тренировочная и валидационная выборка
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
# Обучаем модель XGBoost
model = xgb.train(
    params=xgb_pars, #гиперпараметры модели
    dtrain=dtrain, #обучающая выборка
    num_boost_round=300, #количество моделей в ансамбле
    evals=watchlist, #выборки, на которых считается матрица
    early_stopping_rounds=20, #раняя остановка
    maximize=False, #смена поиска максимума на минимум
    verbose_eval=10 #шаг, через который происходит отображение метрик
)

Предсказать целевой признак на новых данных можно с помощью метода predict():

In [ ]:
#Делаем предсказание на тестовом наборе данных
y_test_predict = np.exp(model.predict(dtest)) - 1
print('Modeling RMSLE %.5f' % model.best_score)

Также как и все модели, основанные на использовании деревьев решений в качестве базовых моделей, XGBoost имеет возможность определения коэффициентов важности факторов. Более того, в библиотеку встроена возможность визуализации важность факторов в виде столбчатой диаграммы. За эту возможность отвечает функция plot_importance():


In [ ]:
fig, ax = plt.subplots(figsize = (15,15))
xgb.plot_importance(model, ax = ax, height=0.5)